In [1]:
from phosphorus import *
%config InteractiveShell.ast_node_interactivity='all'


             _    _                  _    _
            | |  | |                | |  | |
           _| |_ | |__   ___  ___  _| |_ | |__   ___  _ __ _   _  ____
          /     \| '_ \ / _ \/ __|/     \| '_ \ / _ \| '__| | | |/ ___)
         ( (| |) ) | | | (_) \__ ( (| |) ) | | | (_) | |  | |_| ( (__
          \_   _/|_| |_|\___/|___/\_   _/|_| |_|\___/|_|   \__,_|\__ \
            | |                     | |                            _) )
            |_|                     |_|                           (__/

        Welcome to the Phosphorus Meaning Engine v3
        Created by Ezra Keshet (EzraKeshet.com)




In [2]:
from string import ascii_uppercase
DOMAIN = [SemVal.create(c,Type.e) for c in ascii_uppercase]

VACUOUS = object()

def charset(f, domain = None):
  if domain is None:
    domain = DOMAIN
  return {c for c in domain if f(c)}

def iota(f, domain = None):
  return tuple(charset(f,domain))[0]

def single(s):
  return len(s)==1

def empty(s):
  return len(s)==0

def takes(f,x):
  f_type = getattr(f, 'type', [])
  x_type = getattr(x, 'type', None)
  print('TAKES', f, f_type, x, x_type)
  if len(f_type) == 2:
    if f_type[0] == x_type:
      try:
        print(PV(f(x)).eval())
        print(PV(f(x) is not None).eval())
        return PV(f(x) is not None).eval()
      except Exception as e:
        print('Exception in TAKES', e)
        return True
  return False

def is_et(f):
  return getattr(f, 'type', None) == Type.et

class Meaning1(Meaning):
  def rules(self, alpha, a):
    value, rule = None, None
    m = self
    match alpha:
      # TP
      case 't':
        rule = 'TP'
        value = 'a'.e

      # TN
      case str() as s:
        rule = 'TN'
        value = self.lookup(s)
        if value is None:
          value = VACUOUS
      
      # PA
      case ('who', beta):
        rule = 'PA'
        value = (lambda x : m.i(beta, x)).et

      # NN
      case (beta,) if m.i(beta,a) is VACUOUS:
        rule = 'NN'
        value = VACUOUS
        logger.warning('Ignoring vacuous item %s', beta)

      case (beta,):
        rule = 'NN'
        value = PV(m.i(beta,a))

      case (beta, gamma) if  m.i(gamma,a) is VACUOUS:
        rule = 'NN'
        value = PV(m.i(beta,a))
        logger.warning('Ignoring vacuous item %s', gamma)

      case (gamma, beta) if m.i(gamma,a) is VACUOUS:
        rule = 'NN'
        value = PV(m.i(beta,a))
        logger.warning('Ignoring vacuous item %s', gamma)

      # PM
      case (beta, gamma) if is_et(m.i(gamma,a)) and is_et(m.i(beta,a)):
        rule = 'PM'
        value = (lambda x: m.i(beta,a)(x) and m.i(gamma,a)(x)).et

      # FA
      case (beta, gamma) if takes(m.i(beta,a), m.i(gamma,a)):
        rule = 'FA'
        value = PV(m.i(beta,a)(m.i(gamma,a)))
      case (gamma, beta) if takes(m.i(beta,a), m.i(gamma,a)):
        rule = 'AF'
        value = PV(m.i(beta,a)(m.i(gamma,a)))

    return value, rule

m = Meaning1({
    'Bob'       : 'B'.e,
    'invited'   : (lambda y: lambda x: INVITED(x,y)).eet,
    'John'      : 'J'.e,
    'man'       : (lambda x: MAN(x)).et,
    'attended'  : (lambda y: lambda x: ATTENDED(x,y)).eet,
    'the'       : (lambda f: iota(f) if single(charset(f)) else None).et_e,
    '2019_Olympics' : (lambda x: O2019(x)).et,
})


from nltk import Tree, ImmutableTree

tree1 = Tree.fromstring('(S Bob (VP is man))')

tree2 = Tree.fromstring('''
(S John
  (VP attended
    (DP the 2019_Olympics)
  )
)''')

tree3 = Tree.fromstring('''
(CP who
  (S t
    (VP attended
      (DP the 2019_Olympics)
    )
  )
)''')

O2019 = lambda x: 0

console_handler.setLevel(logging.DEBUG)
m.i(tree3, 'M'.e)
console_handler.setLevel(logging.INFO)

visiting M Name(id='M', ctx=Load()) None
CONTEXT {}
Error evaluating M Name(id='M', ctx=Load())
ERROR: name 'M' is not defined
Cleared Memo buffer: {}
Interpreting (CP who (S t (VP attended (DP the 2019_Olympics)))) with parameters: ['M']
visiting lambda x: m.i(beta, x) Lambda(args=arguments(posonlyargs=[], args=[arg(arg='x')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=Call(func=Attribute(value=Name(id='m', ctx=Load()), attr='i', ctx=Load()), args=[Name(id='beta', ctx=Load()), Name(id='x', ctx=Load())], keywords=[])) None
CONTEXT {'m': <__main__.Meaning1 object at 0x70b8ae333340>, 'beta': ImmutableTree('S', ['t', ImmutableTree('VP', ['attended', ImmutableTree('DP', ['the', '2019_Olympics'])])])}
Error evaluating lambda x: m.i(beta, x) Lambda(args=arguments(posonlyargs=[], args=[arg(arg='x')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=Call(func=Attribute(value=Name(id='m', ctx=Load()), attr='i', ctx=Load()), args=[Name(id='beta', ctx=Load()), Name(id='x', ctx=Load())], k

         Interpreting (DP the 2019_Olympics) with parameters: ['x']
            Interpreting 2019_Olympics with parameters: ['x']
visiting lambda x: O2019(x) Lambda(args=arguments(posonlyargs=[], args=[arg(arg='x')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=Call(func=Name(id='O2019', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[])) None
CONTEXT {'O2019': <function <lambda> at 0x70b8e21ba0c0>}
Error evaluating lambda x: O2019(x) Lambda(args=arguments(posonlyargs=[], args=[arg(arg='x')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=Call(func=Name(id='O2019', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[]))
ERROR: invalid syntax (<unknown>, line 1)
visiting O2019(x) Call(func=Name(id='O2019', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[]) None
CONTEXT {'O2019': <function <lambda> at 0x70b8e21ba0c0>}
Error evaluating O2019(x) Call(func=Name(id='O2019', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[])
ERROR: name 'x' is not defined
v

TAKES lambda f: iota(f) if single(charset(f)) else None (('e', 't'), 'e') lambda x: O2019(x) ('e', 't')


CONTEXT {'f': 'lambda x: O2019(x)', 'x': 'lambda x: O2019(x)'}
visiting iota(f) Call(func=Name(id='iota', ctx=Load()), args=[Name(id='f', ctx=Load())], keywords=[]) None
CONTEXT {'f': 'lambda x: O2019(x)', 'x': 'lambda x: O2019(x)'}
Error evaluating iota(f) Call(func=Name(id='iota', ctx=Load()), args=[Name(id='f', ctx=Load())], keywords=[])
ERROR: 'Lambda' object is not callable
visiting iota Name(id='iota', ctx=Load()) None
CONTEXT {'f': 'lambda x: O2019(x)', 'x': 'lambda x: O2019(x)'}
Error evaluating iota Name(id='iota', ctx=Load())
ERROR: invalid syntax (<unknown>, line 1)
visiting f Name(id='f', ctx=Load()) None
CONTEXT {'f': 'lambda x: O2019(x)', 'x': 'lambda x: O2019(x)'}
Evaluated f to lambda x: O2019(x) (<class 'phosphorus.semval.Lambda'>)
CONTEXT {'f': 'lambda x: O2019(x)', 'x': 'lambda x: O2019(x)'}
visiting None Constant(value=None) None
CONTEXT {'f': 'lambda x: O2019(x)', 'x': 'lambda x: O2019(x)'}
Evaluated None to None (<class 'NoneType'>)
CONTEXT {'f': 'lambda x: O2019(

None
False
TAKES lambda x: O2019(x) ('e', 't') lambda f: iota(lambda x: O2019(x)) if single(charset(lambda x: O2019(x))) else None (('e', 't'), 'e')
TAKES lambda y: lambda x: ATTENDED(x, y) ('e', ('e', 't')) None None
TAKES None [] lambda y: lambda x: ATTENDED(x, y) ('e', ('e', 't'))
TAKES x 'e' None None
TAKES None [] x 'e'


lambda x: None

In [3]:
STOP

NameError: name 'STOP' is not defined

In [ ]:
def takes(f,x):
  f_type = getattr(f, 'type', [])
  x_type = getattr(x, 'type', None)
  #print('TAKES', f, f_type, x, x_type)
  if len(f_type) == 2:
    return f_type[0] == x_type
  return False

def is_et(f):
  return getattr(f, 'type', None) == Type.et

class MeaningPV(Meaning):
  def rules(self, alpha, g):
    value, rule = None, None
    m = self
    match alpha:
      # TP
      case str() if '_' in alpha and alpha.split('_')[1].isdecimal():
        i = int(alpha.split('_')[1])
        rule = 'TP'
        value = 'g[i]'.e

      # case int() | str() as i if str(i).isdecimal():
      #   i = int(i)
      #   rule = 'IX'
      #   value = i

      case str() as s:
        rule = 'TN'
        value = self.lookup(s)
      
      # PA
      case (i, beta) if str(i).isdecimal():
        i = int(i)
        rule = 'PA'
        value = (lambda x : m.i(beta, g | {i:x})).et

      # NN
      case (beta,) if m.i(beta,g) is None:
        rule = 'NN'
        value = None
        logger.warning('Ignoring vacuous item %s', beta)

      case (beta,):
        rule = 'NN'
        value = PV(m.i(beta,g))

      case (beta, gamma) if  m.i(gamma,g) is None:
        rule = 'NN'
        value = PV(m.i(beta,g))
        logger.warning('Ignoring vacuous item %s', gamma)

      case (gamma, beta) if m.i(gamma,g) is None:
        rule = 'NN'
        value = PV(m.i(beta,g))
        logger.warning('Ignoring vacuous item %s', gamma)

      # PM
      case (beta, gamma) if is_et(m.i(gamma,g)) and is_et(m.i(beta,g)):
        rule = 'PM'
        value = (lambda x: m.i(beta,g)(x) and m.i(gamma,g)(x)).et

      # FA
      case (beta, gamma) if takes(m.i(beta,g), m.i(gamma,g)):
        rule = 'FA'
        value = PV(m.i(beta,g)(m.i(gamma,g)))
      case (gamma, beta) if takes(m.i(beta,g), m.i(gamma,g)):
        rule = 'AF'
        value = PV(m.i(beta,g)(m.i(gamma,g)))

    return value, rule

m = MeaningPV({
  'snores' : 'lambda x: SNORES(x)'.et,
  'Allie' : 'A'.e,
  'Bob' : 'B'.e,
  'saw' : (lambda y: lambda x: SAW(x,y)).eet
})

m.i(((0, 'he_0'), 'Allie'), {})
#m.i((0, 'he_0'), {})


Interpreting ((0, 'he_0'), 'Allie') with parameters: [{}]
   Interpreting Allie with parameters: [{}]
   => Allie = A 	(TN)
   Interpreting (0, 'he_0') with parameters: [{}]
      Interpreting he_0 with parameters: ['{0: x}']
      => he_0 = x 	(TP)
   => (0, 'he_0') = lambda x: x 	(PA)
=> ((0, 'he_0'), 'Allie') = A 	(FA)


A

In [ ]:
#console_handler.setLevel(logging.DEBUG)
m.i((0, ('Bob', (('is',), ('saw', 't_0')))), {})
#console_handler.setLevel(logging.INFO)

Interpreting (0, ('Bob', (('is',), ('saw', 't_0')))) with parameters: [{}]
   Interpreting ('Bob', (('is',), ('saw', 't_0'))) with parameters: ['{0: x}']
      Interpreting (('is',), ('saw', 't_0')) with parameters: ['{0: x}']
         Interpreting ('saw', 't_0') with parameters: ['{0: x}']
            Interpreting t_0 with parameters: ['{0: x}']
            => t_0 = x 	(TP)
            Interpreting saw with parameters: ['{0: x}']
            => saw = lambda y: lambda x: SAW(x, y) 	(TN)
         => ('saw', 't_0') = lambda z: SAW(z, x) 	(FA)
         Interpreting ('is',) with parameters: ['{0: x}']
            Interpreting is with parameters: ['{0: x}']
            => is = None 	(TN)
Removed vacuous item is
         => ('is',) = None 	(NN)
Removed vacuous item ('is',)
      => (('is',), ('saw', 't_0')) = lambda z: SAW(z, x) 	(NN)
      Interpreting Bob with parameters: ['{0: x}']
      => Bob = B 	(TN)
   => ('Bob', (('is',), ('saw', 't_0'))) = SAW(B, x) 	(AF)
=> (0, ('Bob', (('is',), (

lambda x: SAW(B, x)

In [ ]:
from nltk import Tree, ImmutableTree
tree = Tree.fromstring('(CP 0 (S Bob (VP saw t_0)))')
#console_handler.setLevel(logging.DEBUG)
m.i((0, ('Bob', ('saw', 't_0'))), {})
console_handler.setLevel(logging.INFO)

Interpreting (0, ('Bob', ('saw', 't_0'))) with parameters: [{}]
   Interpreting ('Bob', ('saw', 't_0')) with parameters: ['{0: x}']
      Interpreting ('saw', 't_0') with parameters: ['{0: x}']
         Interpreting t_0 with parameters: ['{0: x}']
         => t_0 = x 	(TP)
         Interpreting saw with parameters: ['{0: x}']
         => saw = lambda y: lambda x: SAW(x, y) 	(TN)
      => ('saw', 't_0') = lambda z: SAW(z, x) 	(FA)
      Interpreting Bob with parameters: ['{0: x}']
      => Bob = B 	(TN)
   => ('Bob', ('saw', 't_0')) = SAW(B, x) 	(AF)
=> (0, ('Bob', ('saw', 't_0'))) = lambda x: SAW(B, x) 	(PA)


PA SAW(B, x)


lambda x: SAW(B, x)

In [ ]:
SNORES = Predicate({('A',), (PV.e('B'),)})
C = PV.e('C')
B = PV.e('B')
B
B == PV.e('B')
console_handler.setLevel(logging.DEBUG)
"(lambda x: (lambda y : SNORES(y))(x))(B)".et
console_handler.setLevel(logging.INFO)

In [ ]:
class MeaningPV(Meaning):
  def rules(self, alpha, g={}):
    value, rule = None, None
    m = self(g)
    match alpha:
      # PA
      case (int() as i, beta):
        rule = 'PA'
        m = self
        value = (lambda x : m.i(beta, g | {i:x})).ee        

      # TP
      case str() if '_' in alpha and alpha.split('_')[1].isdigit():
        i = int(alpha.split('_')[1])
        rule = 'TP'
        value = 'g[i]'.e

      case int() as i:
        rule = 'TN'
        value = i

      case str() as s:
        rule = 'PN'
        value = self.lookup(s)

    return value, rule

m = MeaningPV({
  'snores' : 'lambda x: SNORES(x)'.et,
#  'Allie' : 'A'.e
})

#console_handler.setLevel(logging.DEBUG)
PV(m.i((0, 'he_0'), {}))
console_handler.setLevel(logging.INFO)

In [ ]:
modify = (lambda g,i,a: g | {i:a}).xg
modify
gg = {1:2,3:4}
console_handler.setLevel(logging.DEBUG)
ff = (lambda x : (gg | {1:x})[1]).eg
ff
ff(3)
console_handler.setLevel(logging.INFO)

In [ ]:
class Meaning5b(Meaning):
  def rules(self, alpha):
    value, rule = None, None
    match alpha:
      # PA
      case (int() as i, beta):
        rule = 'PA'
        m = self
        value = (lambda x : (lambda g : g[i])(g | {i:x})).ee
        value = value(m[beta])
        

      # TP
      case str() if '_' in alpha and alpha.split('_')[1].isdigit():
        i = int(alpha.split('_')[1])
        rule = 'TP'
        value = 'g[0]'.e

      case int() as i:
        rule = 'TN'
        value = i

    return value, rule

m = Meaning5b({
  'snores' : 'lambda x: SNORES(x)'.et,
  'Allie' : 'A'.e
})

ff = m[0, 'he_0']
ff
(lambda g : ff).ge({1:'Z'.e, 2:'B'.e})
#ff({1:'Z'.e, 2:'B'.e})('A'.e)

In [ ]:
class Meaning5b(Meaning):
  def rules(self, alpha):
    value, rule = None, None
    match alpha:
      # PA
      case (int() as i, beta):
        rule = 'PA'
        m = self
        value = (lambda y : lambda g: lambda x : y(g|{0:x})).ge_gee
        logger.warning('value context: %s', value.context)
        value = value(m[beta])
        logger.warning('value context: %s', value.context)
        

      # TP
      case str() if '_' in alpha and alpha.split('_')[1].isdigit():
        i = int(alpha.split('_')[1])
        rule = 'TP'
        value = (lambda g : g[0]).ge

      case int() as i:
        rule = 'TN'
        value = i

    return value, rule

m = Meaning5b({
  'snores' : 'lambda x: SNORES(x)'.et,
  'Allie' : 'A'.e
})

ff = m[0, 'he_0']
ff
ff({1:'Z'.e, 2:'B'.e})
ff({1:'Z'.e, 2:'B'.e})('A'.e)

In [ ]:
def modify(g, i, a):
  g[i] = SemVal(str(a), Type.e)
  return g

class Meaning5b(Meaning):
  def rules(self, alpha, g): # pylint: disable=no-self-argument
    """Defines standard rules for combining the meanings of the
    children of a node alpha. Meant to be overridden if different rules are wanted."""

    value, rule = None, None
    m = self(g)
    match alpha:
      # PA
      case (int() as i, beta):
        rule = 'PA'
        m = self
        value = (lambda x : m(modify(g,i,x))[beta]).et

      # PM
      case (beta, gamma) if m[gamma].type == m[beta].type == Type.et:
        rule = 'PM'
        pm_f = Function('lambda f : lambda g: lambda x: f(x) and g(x)', Type.et_et_et)
        value = pm_f(m[beta])(m[gamma])

      # FA
      case (beta, gamma) if  m[gamma] in m[beta].domain() :
        rule = 'FA'
        value = m[beta](m[gamma])
      case (gamma, beta) if  m[gamma] in m[beta].domain() :
        rule = 'AF'
        value = m[beta](m[gamma])

      # NN
      case (beta,):
        rule = 'NN'
        value = m[beta]
      
      # TP
      case str() if '_' in alpha and alpha.split('_')[1].isdigit():
        i = int(alpha.split('_')[1])
        rule = 'TP'
        value = g[i]

      # TN
      case str():
        rule = 'TN'
        value = m.lookup(alpha)

      case int() as i:
        rule = 'TN'
        value = i

    return value, rule

m = Meaning5b({
  'snores' : 'lambda x: SNORES(x)'.et,
  'Allie' : 'A'.e
})

ff = m(['Z'.e, 'B'.e])[(0, ('he_0', 'snores'))]
ff

In [ ]:
class Meaning5a(Meaning):
  def rules(self, alpha, a): # pylint: disable=no-self-argument
    """Defines standard rules for combining the meanings of the
    children of a node alpha. Meant to be overridden if different rules are wanted."""

    value, rule = None, None
    m = self(a)
    match alpha:      
      # PM
      case (beta, gamma) if m[gamma].type == m[beta].type == Type.et:
        rule = 'PM'
        pm_f = Function('lambda f : lambda g: lambda x: f(x) and g(x)', Type.et_et_et)
        value = pm_f(m[beta])(m[gamma])

      # FA
      case (beta, gamma) if  m[gamma] in m[beta].domain() :
        rule = 'FA'
        value = m[beta](m[gamma])
      case (gamma, beta) if  m[gamma] in m[beta].domain() :
        rule = 'AF'
        value = m[beta](m[gamma])

      # NN
      case (beta,):
        rule = 'NN'
        value = m[beta]

      case 't'|'he'|'she'|'it':
        rule = 'TP'
        value = a
      
      # TN
      case str():
        rule = 'TN'
        value = m.lookup(alpha)

    return value, rule

m = Meaning5a({
  'snores' : 'lambda x: SNORES(x)'.et,
})

m('A'.e)['he', 'snores']

In [ ]:
import svgling
from nltk import Tree

tree = Tree.fromstring('(CP 1 (S (NP (DT The_1) (NN cat)) (VP (VBZ eats) (NP (DT the) (NN mouse)))))')

def split_leaf(node):
  if isinstance(node, str):
    l = node.split('_')
    if len(l) > 1:
      return svgling.core.subscript_node(l[0], l[1]), ()
    return node, ()

def _repr_svg_(self):
  return svgling.draw_tree(self, tree_split=split_leaf)._repr_svg_()

Tree._repr_svg_ = _repr_svg_
tree
type(tree[0][0][0])


In [ ]:
from string import ascii_uppercase
DOMAIN = [SemVal.create(c,Type.e) for c in ascii_uppercase]
A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z = DOMAIN

def charset(f, domain = None):
  #logger.warning(f'CHARSET of {repr(f)}')
  if domain is None:
    domain = DOMAIN
  out = {c for c in domain if f(c)}
  #logger.warning(f'CHARSET of {f} is {out}')
  return out

def iota(f, domain = None):
  #logger.warning(f'IOTA of {f}')
  return max(charset(f,domain))

In [ ]:
from IPython.display import HTML
from nltk import Tree
# Domain and Predicates / Relations
A, B, C, D, E, F, G = DOMAIN[:7]
PAWN = Predicate({
    (A,), (B,), (C,), (D,), (E,), (F,), (G,)
})
LEFT_OF = Predicate({
    (A,B), (A,C), (A,D), (A,E), (A,F), (A,G),
    (B,C), (B,D), (B,E), (B,F), (B,G),
    (C,D), (C,E), (C,F), (C,G),
    (D,E), (D,F), (D,G),
    (E,F), (E,G),
    (F,G)            
})
BLACK = Predicate({
    (B,), (D,), (E,), (F,)
})


# Lexical entries
m3 = Meaning({
    'leftmost.et'  : 'lambda x : len(charset(lambda y : LEFT_OF(y,x)))==0'.et,
    'pawn'      : 'lambda x : PAWN(x)'.et,
    'the'       : 'lambda f : (len(charset(f))==1, iota(f))'.et_e,
    'black'     : 'lambda x : BLACK(x)'.et
})
    

# CHANGE ONLY BELOW
m3.update({
    'leftmost.et_et' : 'lambda f : lambda x: len(charset(lambda y : f(y) and LEFT_OF(y,x)))==0'.et_et
})
# CHANGE ONLY ABOVE

HTML("<h3>[black pawn]</h3>")
charset(m3['black','pawn'])

HTML("<h3>[leftmost.et pawn]</h3>")
charset(m3['leftmost.et','pawn'])

tree3 = Tree.fromstring('(NP leftmost.et (NP black pawn))')
HTML('<h3>[leftmost.et black pawn]</h3>')
tree3
charset(m3[tree3])

HTML("<h3>[leftmost.et_et pawn]</h3>")
charset(m3['leftmost.et_et','pawn'])

tree3 = Tree.fromstring('(NP leftmost.et_et (NP black pawn))')
HTML('<h3>[leftmost.et_et black pawn]</h3>')
tree3
charset(m3[tree3])

In [ ]:
from nltk import ImmutableTree
tt = ImmutableTree.fromstring('(NP leftmost.et_et (NP black pawn))')
type([tt]) == list


In [ ]:
console_handler.setLevel(logging.INFO)
the = 'lambda f: iota(f) if len(charset(f)) == 1 else None'.et_e
dog = 'lambda x: x in DOMAIN[:1]'.et
charset(dog)
the(dog)

In [ ]:
#DOMAIN = DOMAIN[:3]
#DOMAIN
KILLED = Predicate({(A,B),})
BLACK = Predicate({(B,), (C,)})
CAT = Predicate({(B,)})
ESCAPED = Predicate({(A,)})

del KILLED
del BLACK
del CAT
del ESCAPED

In [ ]:
m= Meaning()
m.update({
  'killer'  : 'lambda y: lambda x :KILLED(x,y)'.eet,
  'black'   : 'lambda x: BLACK(x)'.et,
  'cat'     : 'lambda x: CAT(x)'.et,
  'escaped' : 'lambda x: ESCAPED(x)'.et,
  'the'     : 'lambda f: ( len(charset(f))==1, iota(f) )'.et_e
})

from nltk import Tree
tree = Tree.fromstring("""
(S  (DP the
        (NP killer
            (PP of
                (DP the
                    (NP black cat)
                )
            )
        )
    )
    (VP escaped)
)
""")

test = tree[0][1][1][1]
test = tree
test

m[test]




In [ ]:
m.memo

In [ ]:
from nltk import Tree
tree = Tree.fromstring("""
(S
  (NP Julius)
  (VP is
      (VP sleeping
          (PP on (NP the couch))
      )
  )
)""")
tree

m = Meaning({
    'Julius'    : 'J'.e,
    'the'       : 'lambda f: iota(f)'.et_e,

})

#m[tree]

In [ ]:
#!pip install nltk
from nltk import Tree
#!pip install svgling

In [ ]:
node = parse('PAWN("A")', mode='eval')
obj = compile(node, '<string>', 'eval')
eval(obj)

In [ ]:
m = Meaning({
    'Joe'     : 'J'.e,
    'Kaline'  : 'K'.e,
    'cat'     : 'lambda x: CAT(x)'.et,
    'gray'    : 'lambda f : lambda x: f(x) and GRAY(x)'.et_et,
})

m['Joe', 'cat']

In [ ]:
t = Tree.fromstring('(S (NP Joe) (VP cat (VP (V is) (NP cat gray))))')
t

m[t]

In [ ]:
m.update({
  'gray'    : 'lambda f : lambda x: f(x) and GRAY(x)'.eet_t,
  })

m['gray']
m[t]

In [ ]:
class Meaning4(Meaning):
  def rules(m, alpha):
    match alpha:      # Note: m.quiet(  ) turns off printing
      # PM
      case (beta, gamma) if m.quiet(  m[gamma].type == m[beta].type == Type.et ):
        rule = 'PM'
        value = 'lambda f : lambda g: lambda x: f(x) and g(x)'.et_et_et(m[beta])(m[gamma])

      # FA
      case (beta, gamma) if m.quiet(  m[gamma] in m[beta].domain()  ):
        rule = 'FA'
        value = m[beta](m[gamma])
      case (gamma, beta) if m.quiet(  m[gamma] in m[beta].domain()  ):
        rule = 'AF'
        value = m[beta](m[gamma])

      # NN
      case (beta,):
        rule = 'NN'
        value = m[beta]

      # TN
      case str():
        rule = 'TN'
        value = m.lookup(alpha)

      case _:
        rule = '??'
        value = alpha

    return value, rule

m4 = Meaning4({
    'Joe'     : 'J'.e,
    'Kaline'  : 'K'.e,
    'Texas'   : 'T'.e,
    'in'      : 'lambda x: lambda y: IN(y,x)'.eet,
    'cat'     : 'lambda x: CAT(x)'.et,
    'gray'    : 'lambda x: GRAY(x)'.et,
})

m4['gray', 'cat']
m4['Kaline', [['gray', 'cat'], ['in', 'Texas']]]

In [ ]:
class Meaning3(Meaning):
  def rules(m, alpha):
    match alpha:      # Note: m.quiet(  ) turns off printing
      # FA
      case (beta, gamma) if m.quiet(  m[gamma] in m[beta].domain()  ):
        rule = 'FA'
        value = m[beta](m[gamma])
      case (gamma, beta) if m.quiet(  m[gamma] in m[beta].domain()  ):
        rule = 'AF'
        value = m[beta](m[gamma])

      # NN
      case (beta,):
        rule = 'NN'
        value = m[beta]

      # TN
      case str():
        rule = 'TN'
        value = m.lookup(alpha)

      case _:
        rule = '??'
        value = alpha

    return value, rule

m3 = Meaning3({
    'Joe'     : 'J'.e,
    'Kaline'  : 'K'.e,
    'cat'     : 'lambda x: CAT(x)'.et,
    'gray'    : 'lambda f: lambda x: f(x) and GRAY(x)'.et_et,
    'fond'    : ...
})

#m3['Joe']
#m3['Kaline']
#m3['cat']
#m3['Kaline', 'cat']
m3[('a',)]
m3['a', 'cat']
m3['is', ['a', 'cat']]
m3['Kaline', [['fond', 'Joe'], ['gray', 'cat']]]
m3['Kaline', 'gray']

In [ ]:
m3.update({'is': 'lambda f: lambda x: f(lambda y:1)(x)'.et_et__et})

m3['gray']
m3['is', 'gray']

In [ ]:
tree = Tree.fromstring('(S (NP (N Kaline)) (VP is (NP (Det a) (N cat))))')
tree
m3[tree]

In [ ]:
class Meaning3(Meaning):
  def rules(m, alpha):
    match alpha:      # Note: m.quiet(  ) turns off printing
      # FA
      case (beta, gamma) if m.quiet(  m[gamma] in m[beta].domain()  ):
        rule = 'FA'
        value = m[beta](m[gamma])
      case (gamma, beta) if m.quiet(  m[gamma] in m[beta].domain()  ):
        rule = 'AF'
        value = m[beta](m[gamma])

      # NN
      case (beta,):
        rule = 'NN'
        value = m[beta]

      # TN
      case str():# if alpha in m:
        rule = 'TN'
        value = m.lookup(alpha)

      case _:
        rule = '??'
        value = alpha
    return value, rule

m = Meaning3({
    'Ann'     : 'A'.e,
    'smokes'  : 'lambda x: SMOKES(x)'.et,
    'tall'    : 'lambda x: TALL(x)'.et,
    'likes'   : 'lambda x: lambda y: LIKES(x,y)'.eet,
#    'is'      : None,
})

m['Ann']
m['smokes']
m['Ann', 'smokes']
m['smokes', 'Ann']

In [ ]:
m['tall', 'is']

In [ ]:
t = [1,2]
t = t - [1]

In [ ]:
'lambda f:lambda x: f(x) and GRAY(x)'.et

In [ ]:
'(lambda x: lambda y: lambda z: INTRODUCED(z,x,y))(S)'.eet

a. [λx.[λy.[λz.z introduced x to y]]](Ann)(Sue)

b. [λx.[λy.[λz.z introduced x to y](Ann)](Sue)]

c. [λx.[λy.[λz.z introduced x to y](Ann)]](Sue) 

d. [λx.[λy.[λz.z introduced x to y]](Ann)](Sue)

e. [λf.[λx.f(x)=1 and x is gray]]([λy.y is a cat])

f. [λg.[λx.g(x)(Ann)=1]]([λy.[λz.z saw y]])

g. [λm.[λn.n>3 and n<7](m)] 

h. [λm.[λn.[λo.o>3 and o<7](n)](m)] 

In [ ]:
'lambda g: (lambda x:g(x)(Ann))'.eet_et('lambda y:lambda z:SAW(z,y)'.eet)

'lambda m: lambda n:n>3 and m<7'.eet('3'.e)

In [ ]:
catf = 'lambda y:CAT(y)'.et

ff = 'lambda f: lambda x: f(x) and GRAY(x)'.et_et
ff(catf)

gg = 'lambda f: f(A)'.eet_et
gg
gg('lambda x:lambda y:FOO(x,y)'.eet)

In [ ]:
Function('lambda y: CAT(y)', Type(('e', 't')))('x') and GRAY('x')

In [ ]:
import ast
ast.dump(ast.parse('x smokes', mode= 'eval'))